In [53]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import csv


In [2]:
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)

# Generate URLs to scrape
base_url = "https://weworkremotely.com/top-trending-remote-jobs?page="
urls = [f"{base_url}{i}" for i in range(1, 4)]  # scrape pages 1 to 3, extend range to scrape more pages

all_job_links = []

for index, url in enumerate(urls):
    driver.get(url)
    print(f"Scraping page {index + 1}")

    # Accept cookies on the first page only if necessary
    if index == 0:
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'adroll_button_text'))).click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'adroll_consent_allow_all'))).click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'fc-button'))).click()
        except:
            print("Cookie buttons not found or already accepted.")

    # Scroll to bottom to load all info
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(3, 6))

    # Scrape job links
    job_links = driver.find_elements(By.CSS_SELECTOR, "section.jobs li > a")
    print(f"Found {len(job_links)} job listings on page {index + 1}")

    # Extract hrefs and filter out website url
    page_links = [
        link.get_attribute("href")
        for link in job_links
        if link.get_attribute("href") and link.get_attribute("href") != "https://weworkremotely.com/"
    ]

    all_job_links.extend(page_links)

    # Random delay between pages
    time.sleep(random.uniform(2, 5))

print(f"\n Total unique job links scraped: {len(all_job_links)}")

driver.quit()

Scraping page 1
Cookie buttons not found or already accepted.
Found 51 job listings on page 1
Scraping page 2
Found 51 job listings on page 2
Scraping page 3
Found 51 job listings on page 3

 Total unique job links scraped: 150


In [3]:
all_job_links

['https://weworkremotely.com/remote-jobs/dataannotation-tech-ft-pt-remote-ai-prompt-engineering-evaluation-will-train',
 'https://weworkremotely.com/remote-jobs/prowritersites-freelance-writer',
 'https://weworkremotely.com/remote-jobs/clipboard-health-client-support-specialist-healthcare-facilities-b2b-1',
 'https://weworkremotely.com/remote-jobs/usedconex-com-remote-sales-chat-representative-3',
 'https://weworkremotely.com/remote-jobs/used-conex-llc-entry-level-sales-agent-chat-only-side-hustle',
 'https://weworkremotely.com/remote-jobs/native-camp-online-english-teacher-30',
 'https://weworkremotely.com/remote-jobs/new-law-business-model-marketing-automation-saas-services-line-of-business-owner',
 'https://weworkremotely.com/remote-jobs/the-ward-law-group-virtual-assistants-for-a-personal-injury-law-firm-customer-service',
 'https://weworkremotely.com/remote-jobs/spiralyze-ui-ux-designer',
 'https://weworkremotely.com/remote-jobs/xm-cloud-security-engineer',
 'https://weworkremotel

In [13]:
test_links = all_job_links[0:4]

In [54]:
# Setup Chrome options
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)


# Data container
job_data = []

for i, link in enumerate(test_links):
    driver.get(link)

    if i == 0:
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'adroll_consent_button'))).click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'adroll_consent_allow_all'))).click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'fc-cta-consent'))).click()
        except:
            print('No cookie button found')

    try:
        job_title = driver.find_element(By.CLASS_NAME, 'lis-container__header__hero__company-info__title').text
        company_block = driver.find_element(By.CLASS_NAME, 'lis-container__job__sidebar__companyDetails__info')
        company_title = company_block.find_element(By.CLASS_NAME, 'lis-container__job__sidebar__companyDetails__info__title').text

        try:
            jobs_posted_text = WebDriverWait(company_block, 30).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'lis-container__job__sidebar__companyDetails__info__jobs-posted'))
            ).text.strip()
        except:
            jobs_posted_text = "NA"

        # Job metadata
        job_block = driver.find_element(By.CLASS_NAME, 'lis-container__job__sidebar__job-about')
        category_items = job_block.find_elements(By.CLASS_NAME, 'lis-container__job__sidebar__job-about__list__item')

        job_details = {}
        for item in category_items:
            lines = item.text.strip().split("\n")
            if len(lines) >= 2:
                label = lines[0].strip()
                value = "\n".join(lines[1:]).strip()
                job_details[label] = value

        # Extract desired fields
        job_entry = {
            "Ranking": i + 1,
            "Job Title": job_title,
            "Company": company_title,
            "Jobs Posted": jobs_posted_text,
            "Job Type": job_details.get("Job type", "NA"),
            "Category": job_details.get("Category", "NA"),
            "Salary": job_details.get("Salary", "NA"),
            "Region": job_details.get("Region", "NA"),
            "Country": job_details.get("Country", "NA"),
            "Link": link
        }

        job_data.append(job_entry)

    except Exception as e:
        print(f"❌ Error on job {i+1}: {e}")

    time.sleep(1)  # optional delay between requests

driver.quit()

# Write to CSV
with open("top_trending_jobs.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=job_data[0].keys())
    writer.writeheader()
    writer.writerows(job_data)

print(f"✅ Scraped {len(job_data)} jobs and saved to 'top_trending_jobs.csv'")


✅ Scraped 4 jobs and saved to 'top_trending_jobs.csv'
